In [1]:
from google.colab import drive
drive.mount("/content/drive")
!source /content/drive/MyDrive/colab_env/bin/activate
%cd /content/drive/MyDrive/applied_DL/

Mounted at /content/drive
/content/drive/MyDrive/applied_DL


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

from TimeSeriesDiffusion.data.get_data import get_dataset
from TimeSeriesDiffusion.trainer import trainer
from TimeSeriesDiffusion.models.unet import Unet
from TimeSeriesDiffusion.models.convnet import ConvNetWithTimeEmbeddings
from TimeSeriesDiffusion.diffusion_model.diffusion_ingridients import DiffusionIngridients
from TimeSeriesDiffusion.diffusion_model.inference import generative_inference

# Define arguments
class Args:
    def __init__(self):
        self.dataset = "sine_wave"  # Change this to your dataset name
        self.data_path = "TimeSeriesDiffusion/raw_data_csv"  # Change this to the path of your data
        self.optimizer = "adam"
        self.batch_size = 16
        self.learning_rate = 0.01
        self.epochs = 5
        self.time_steps = 10
        self.beta_lower = 0.01
        self.beta_upper = 0.99
        self.scheduler = "linear"
        self.noise = "gaussian"
        self.loss = "l1"
        self.dim = 128
        self.num_samples = 2
        self.csv_path = "results_csv_and_plots"
        self.noisy_file_name = "noisy"
        self.gen_file_name = "gen"
        self.log_schedule = 5
        self.anno_threshold = 0.1
        self.model_arch = 'convnet'

args = Args()

device = "cuda" if torch.cuda.is_available() else "cpu"

full_root = args.data_path
train_data, test_data = get_dataset(dataset=args.dataset, root=full_root)

train_loader = DataLoader(train_data,
                          batch_size=args.batch_size,
                          shuffle=True,
                          num_workers=1,
                          pin_memory=True,
                          drop_last=True)

test_loader = DataLoader(test_data,
                         batch_size = args.batch_size,
                         shuffle = False,
                         num_workers = 1,
                         pin_memory = True)

for batch in train_loader:
    x, _ = batch
    break

if args.model_arch == 'convnet':
  model = ConvNetWithTimeEmbeddings(dim = 32)
elif args.model_arch == 'unet':
  model = Unet(dims = 32)

model = nn.DataParallel(model)

diff = DiffusionIngridients(
    model=model,
    time_steps=args.time_steps,
    beta_lower=args.beta_lower,
    beta_upper=args.beta_upper,
    scheduler=args.scheduler,
    noise=args.noise,
    loss=args.loss,
    device=device,
)

if not os.path.exists("results"):
    os.makedirs("results")

if not os.path.exists(args.csv_path):
    os.makedirs(args.csv_path)

writer = SummaryWriter(comment="_pretraining")
t0 = time.time()

# training loop
for epoch in range(1, args.epochs + 1):
    train_loss = trainer(
        data_loader=train_loader,
        optimizer=args.optimizer,
        batch_size=args.batch_size,
        learning_rate=args.learning_rate,
        epochs=args.epochs,
        time_steps=args.time_steps,
        loss=args.loss,
        diffusion_model=diff,
    )
    writer.add_scalar("Train/Loss", train_loss, epoch)

    if epoch % 1 == 0:
        torch.save(
            model.module.state_dict(), "results/pretraining_{}.pth".format(epoch)
        )

        # generate result
        a = generative_inference(
            diffusion_model=diff,
            test_loader=test_loader,
            num_samples=args.num_samples,
            csv_path=args.csv_path,
            generated_file_name=args.gen_file_name,
            noisy_file_name=args.noisy_file_name,
            device=device,
            noise=args.noise,
            log_schedule=args.log_schedule,
            anno_threshold=args.anno_threshold,
            epoch=epoch
        )
        print(a)

t1 = time.time() - t0
print("Time elapsed in hours:", t1 / 3600)

100%|██████████| 3/3 [00:00<00:00,  8.73it/s, Loss=2.31]


Running generative inference for epoch: 1
Sample data loaded
Test data shape: torch.Size([31, 16])
Test labels shape: torch.Size([31])


sampling loop time step: 100%|██████████| 10/10 [00:00<00:00, 6973.07it/s]
/content/drive/MyDrive/applied_DL/TimeSeriesDiffusion/diffusion_model/generator_helpers.py:174: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  plt.tight_layout()
/content/drive/MyDrive/applied_DL/TimeSeriesDiffusion/diffusion_model/generator_helpers.py:192: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  plt.tight_layout()


hello there


100%|██████████| 3/3 [00:00<00:00, 27.85it/s, Loss=2.67]


...

hello there


100%|██████████| 3/3 [00:00<00:00, 24.08it/s, Loss=1.93]


Running generative inference for epoch: 5
Sample data loaded
Test data shape: torch.Size([31, 16])
Test labels shape: torch.Size([31])


sampling loop time step: 100%|██████████| 10/10 [00:00<00:00, 10776.73it/s]


hello there
Time elapsed in hours: 0.02795210745599535


In [ ]:
from google.colab import drive
drive.mount('/content/drive')